In [15]:
import sys
import os
import matplotlib.pyplot as plt

# Add the path to utils.py
sys.path.append(r"c:\Document\Serieux\Travail\python_work\cEBNM_torch\py")
from distribution_operation import *
from utils import *
from numerical_routine import *
from posterior_computation import *
from ash import *

In [ ]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])

res= ash(betahat, sebetahat, mult=np.sqrt(2))
print(res.log_lik)
print(res.pi )
print(res.scale     )
mult=2



scale=autoselect_scales_mix_norm(betahat  = betahat,
                                         sebetahat= sebetahat,
                                         mult=np.sqrt(2))
location=0*scale+2

In [ ]:
L= get_data_loglik_normal(betahat=betahat ,
                                 sebetahat=sebetahat ,
                                 location=0*scale,
                                 scale=scale)
optimal_pi = optimize_pi( np.exp(L),
                                 penalty=10,
                                 verbose=True) 
optimal_pi
 

In [ ]:
location

In [19]:
data_loglik = get_data_loglik_normal (  betahat,sebetahat, location, scale)

In [ ]:
log_pi =np.log(optimal_pi+1e-32)

optimal_pi 

In [21]:

L= get_data_loglik_normal(betahat=betahat ,
                                 sebetahat=sebetahat ,
                                 location=0*scale,
                                 scale=scale)
optimal_pi = optimize_pi( np.exp(L),
                                 penalty=10,
                                 verbose=True) 
 

In [ ]:

log_pi =np.log(optimal_pi+1e-32)

optimal_pi 

In [23]:
data_loglik = get_data_loglik_normal (  betahat,sebetahat, location, scale)
log_post_assignment = apply_log_sum_exp(data_loglik, log_pi)
t_ind_Var =np.zeros((betahat.shape[0], scale.shape[0]))
 
for i in range(t_ind_Var.shape[0]):
        t_ind_Var[i , ]= np.concatenate(
                                        ([0], 
                                         1/((1/sebetahat[i]**2)+ (1/scale[1:]**2)) )
                                        )#assume that first entry of scale is 0
        
temp=np.zeros((betahat.shape[0], scale.shape[0]))

for i in range(temp.shape[0]):
        temp[i,] = (t_ind_Var[i,]/(sebetahat[i]**2))*(betahat[i] )+ location*(1-t_ind_Var[i,]/(sebetahat[i]**2))

post_mean  = np.sum ( np.exp(log_post_assignment)* temp, axis=1)
post_mean2 = np.sum ( np.exp(log_post_assignment)*( t_ind_Var+ temp**2), axis=1)
post_sd    = np.sqrt(post_mean2-post_mean**2) 

In [ ]:
post_mean

In [25]:
data_loglik = get_data_loglik_normal (  betahat,sebetahat, location, scale)
log_post_assignment = apply_log_sum_exp(data_loglik, log_pi)
t_ind_Var =np.zeros((betahat.shape[0], scale.shape[0]))
 
for i in range(t_ind_Var.shape[0]):
        t_ind_Var[i , ]= np.concatenate(
                                        ([0], 
                                         1/((1/sebetahat[i]**2)+ (1/scale[1:]**2)) )
                                        )#assume that first entry of scale is 0
        
temp=np.zeros((betahat.shape[0], scale.shape[0]))

for i in range(temp.shape[0]):
        temp[i,] = (t_ind_Var[i,]/(sebetahat[i]**2))*(betahat[i] )+ location*(1-t_ind_Var[i,]/(sebetahat[i]**2))

post_mean  = np.sum ( np.exp(log_post_assignment)* temp, axis=1)
post_mean2 = np.sum ( np.exp(log_post_assignment)*( t_ind_Var+ temp**2), axis=1)
post_sd    = np.sqrt(post_mean2-post_mean**2) 

In [ ]:
post_mean

In [ ]:
plt.scatter(post_mean, betahat)
res= ash(betahat,sebetahat, mode=2)
plt.scatter(res.post_mean, betahat)

In [ ]:
scale=autoselect_scales_mix_norm(betahat  = betahat,
                                         sebetahat= sebetahat,
                                         mult=np.sqrt(2))

location=scale
print (location)
if location is None:
    print(1)
else:
    print(2)

location=None
location is None